In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/training.1600000.processed.noemoticon.csv",names=['data','ids','date','flag','user','text'],encoding="ISO-8859-1")

In [28]:
df.head(2)

,data,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [29]:
!pip install tensorflow
!pip install transformers

In [105]:
df=df.loc[1:10000] #only 10k tweets were taken because of huge time consumption.Kindly chech it dear mentor.i wanted to do something new via deep learning,therefore didnt try machine learning techniques

In [106]:

box=[]
from transformers import pipeline
classifier=pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment")
for fea in df['text']:
  box.append(classifier(fea))

In [107]:
 len(box)

10000

In [108]:
label=[]
score=[]
for fea in box:
  for features in fea:
    for extraction in features:
      if type(features[extraction])== str:
        label.append(features[extraction])
      elif type(features[extraction])==float:
        score.append(features[extraction])
print(len(label))
print(len(score))

10000
10000


In [109]:

for fea in range(10000):
  if label[fea]=='LABEL_0':
    label[fea]=2
  elif label[fea]=='LABEL_1':
    label[fea]=0
  elif label[fea]=='LABEL_2':
    label[fea]=4



In [110]:
df['label']=label
df['score']=score

In [36]:
df.head()

,data,ids,date,flag,user,text,label,score
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,2,0.932836
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0,0.702750
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,2,0.813473
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",2,0.945602
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,0,0.656295


In [37]:
df.shape

(10000, 8)

In [38]:
df1=df.drop(['ids','data','date','flag','user','score'],axis=1)
df1

,text,label
1,is upset that he can't update his Facebook by ...,2
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,2
4,"@nationwideclass no, it's not behaving at all....",2
5,@Kwesidei not the whole crew,0
...,...,...
9996,stupid dvds stuffing up the good bits in jaws.,2
9997,@Dandy_Sephy No. Only close friends and family...,2
9998,CRAP! After looking when I last tweeted... WHY...,2
9999,Its Another Rainboot day,4


In [39]:
df1.shape

(10000, 2)

In [111]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [112]:
ps=PorterStemmer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [113]:
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

In [114]:
def remove_url(text):
  pattern=re.compile(r'https?://\S+|WWW\.\S+')
  return pattern.sub(r'',text)

In [115]:
import string,time
string.punctuation
exclude=string.punctuation

def remove_punc(text):
  for char in exclude:
    text=text.replace(char,'')
  return text

In [116]:
def remove_stopwords(text):
  new_text=[]

  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)

  x=new_text[:]
  new_text.clear()
  return " ".join(x)

In [117]:
tex=[]
for fea in df1['text']:
  fea=fea.lower()
  tex.append(fea)
df1['text']=tex
df1['text']

1        upset cant updat facebook text might cri resul...
2        kenichan dive mani time ball manag save 50 res...
3                          whole bodi feel itchi like fire
4                    nationwideclass behav im mad cant see
5                                      kwesidei whole crew
                               ...                        
9996                          stupid dvd stuf good bit jaw
9997     dandysephi close friend famili im afraid im wo...
9998        crap look last tweet earli 10 shouldnt like 12
9999                                    anoth rainboot day
10000    think there problem isp area somethingmi conne...
Name: text, Length: 10000, dtype: object

In [118]:
df1['text']=df1['text'].apply(remove_html_tags)
df1['text']=df1['text'].apply(remove_url)
df1['text']=df1['text'].apply(remove_punc)
df1['text']=df1['text'].apply(remove_stopwords)


In [119]:
def stem_words(text):
  return " ".join([ps.stem(word) for word in text.split()])

In [120]:
df1['text']=df1['text'].apply(stem_words)

In [121]:
df1

,text,label
1,upset cant updat facebook text might cri resul...,2
2,kenichan dive mani time ball manag save 50 res...,0
3,whole bodi feel itchi like fire,2
4,nationwideclass behav im mad cant see,2
5,kwesidei whole crew,0
...,...,...
9996,stupid dvd stuf good bit jaw,2
9997,dandysephi close friend famili im afraid im wo...,2
9998,crap look last tweet earli 10 shouldnt like 12,2
9999,anoth rainboot day,4


In [131]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten


In [123]:
from tensorflow.keras.layers import Dropout


In [124]:
### Vocabulary size
voc_size=5000

In [125]:
onehot_repr=[one_hot(words,voc_size)for words in df1['text']]
onehot_repr

[[4105, 4135, 201, 1934, 3772, 4712, 644, 614, 3884, 3163, 2208, 4328],
 [3441, 3006, 3144, 1203, 1810, 1630, 1465, 3462, 4921, 502, 4059],
 [311, 331, 3277, 1739, 187, 1027],
 [3812, 1302, 876, 1228, 4135, 2839],
 [1739, 311, 864],
 [3903, 575],
 [3907, 3036, 262, 1203, 2839, 1056, 327, 292, 292, 2712, 876, 489, 1284],
 [1747, 3762, 3603],
 [1962, 1066, 4070],
 [2345, 2449, 15, 854, 4241],
 [4414, 3274],
 [1241, 1632, 2425, 637, 4528, 2832, 746, 692],
 [2860, 3869, 4832, 4175, 3064, 3361, 1807],
 [4198, 1102, 2910, 3603, 4858, 4190, 726, 1391, 2522, 2089],
 [3678, 43, 1154, 637, 1685, 957, 3908],
 [2788, 4391, 1736, 3500, 4650, 637, 2239, 2259, 2313, 4640],
 [4853, 3946],
 [4546, 3435, 2342, 559, 3278, 2839, 2626, 2140, 2436],
 [3829, 1636, 398, 1791, 570, 2838, 1791],
 [2378, 1160, 3603, 2607, 965, 765],
 [4240, 3880, 2496, 63, 1903, 4286, 1849, 712, 2342, 1203, 2768],
 [3292, 1925, 2736, 2111],
 [920, 502, 3457],
 [3528, 642, 1747, 3744],
 [2331, 2496, 3507, 2198],
 [502, 644, 4145,

In [126]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3163 2208 4328]
 [   0    0    0 ... 4921  502 4059]
 [   0    0    0 ... 1739  187 1027]
 ...
 [   0    0    0 ... 2143  187  567]
 [   0    0    0 ... 2970 4286 1160]
 [   0    0    0 ... 2397  911 3163]]


In [140]:
## Creating model
embedding_vector_features=50 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,return_sequences=True))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))
model.add(Flatten())
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 20, 50)            250000    
                                                                 
 lstm_16 (LSTM)              (None, 20, 128)           91648     
                                                                 
 dropout_10 (Dropout)        (None, 20, 128)           0         
                                                                 
 lstm_17 (LSTM)              (None, 20, 64)            49408     
                                                                 
 dropout_11 (Dropout)        (None, 20, 64)            0         
                                                                 
 lstm_18 (LSTM)              (None, 20, 32)            12416     
                                                                 
 flatten_1 (Flatten)         (None, 640)             

In [83]:
len(embedded_docs)

10000

In [141]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(df['label'])

In [142]:
X_final.shape,y_final.shape

((10000, 20), (10000,))

In [151]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33)

In [153]:
### Finally Training
model.fit(X_train,y_train,validation_split=0.2,epochs=5,batch_size=64)

Epoch 1/5
84/84 [==============================] - 10s 120ms/step - loss: 0.0909 - accuracy: 0.9767 - val_loss: 0.0922 - val_accuracy: 0.9761
Epoch 2/5
84/84 [==============================] - 9s 106ms/step - loss: 0.0620 - accuracy: 0.9847 - val_loss: 0.0972 - val_accuracy: 0.9731
Epoch 3/5
84/84 [==============================] - 11s 134ms/step - loss: 0.0441 - accuracy: 0.9888 - val_loss: 0.1044 - val_accuracy: 0.9716
Epoch 4/5
84/84 [==============================] - 8s 96ms/step - loss: 0.0351 - accuracy: 0.9912 - val_loss: 0.1083 - val_accuracy: 0.9709
Epoch 5/5
84/84 [==============================] - 11s 132ms/step - loss: 0.0282 - accuracy: 0.9940 - val_loss: 0.1166 - val_accuracy: 0.9701


In [154]:
model.evaluate(X_test,y_test)

104/104 [==============================] - 2s 19ms/step - loss: 0.1799 - accuracy: 0.9573


[0.1798592060804367, 0.9572727084159851]